In [35]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json
import pymysql

### 한국은행 OpenAPI 조회 서비스 사용 
http://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=main

In [45]:
"""
한국은행 openAPI를 활용하여
경기동행지수 받아옴
"""
def getEcosAPI(API_KEY, code, max, start_month, end_month):
    url = 'http://ecos.bok.or.kr/api/StatisticSearch/%s/json/kr/1/%s/%s/MM/%s/%s/?/?/?/' % (API_KEY, max, code, start_month, end_month)
    result = urlopen(url)
    html = result.read()
    
    return json.loads(html)


In [51]:
API_KEY = "YOUR_API_KEY"

data = getEcosAPI(API_KEY, code = '085Y026', max = 100000, start_month = '200807', end_month = '202010')

### 경기동행지수 순환변동치 구하기
ITEM_CODE1 = 'I16D'

In [52]:
data = data["StatisticSearch"]["row"]
produce = pd.DataFrame(data)

In [53]:
cci = produce[produce['ITEM_CODE1'] == 'I16D']
cci

,UNIT_NAME,STAT_NAME,ITEM_CODE1,STAT_CODE,ITEM_CODE2,ITEM_CODE3,ITEM_NAME1,ITEM_NAME2,DATA_VALUE,ITEM_NAME3,TIME
584,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,100.9,,200807
585,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,100.8,,200808
586,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,100.8,,200809
587,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,100.5,,200810
588,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,99.7,,200811
...,...,...,...,...,...,...,...,...,...,...,...
725,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,97.6,,202004
726,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,96.8,,202005
727,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,97,,202006
728,2015=100,16.1.4 경기종합지수,I16D,085Y026,,,동행지수순환변동치,,97.2,,202007


In [54]:
cci = cci.loc[:, ['TIME', 'DATA_VALUE']].reset_index(drop=True)
cci

,TIME,DATA_VALUE
0,200807,100.9
1,200808,100.8
2,200809,100.8
3,200810,100.5
4,200811,99.7
...,...,...
141,202004,97.6
142,202005,96.8
143,202006,97
144,202007,97.2


### DB연동

In [67]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

### DB table 생성

In [68]:
"""
Coincident Composite Index 저장할 table 생성
"""
create_table_query = """
CREATE TABLE IF NOT EXISTS coincident_composite_index(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    cci double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)
# charset=utf8mb4; 유니코드 스트링 저장

0

In [69]:
cur.execute("desc coincident_composite_index")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,cci,double,YES,,None,


### CCI 월별로 저장

In [70]:
"""
날짜에서 year, month 추출
"""
def extract_year_and_month(period):
    year = period[0:4]
    month = period[4:6]
    return year, month


In [74]:
"""
db table에 저장
"""
for i in range(len(cci)):
    query = """ 
    Insert into coincident_composite_index (year, month, cci) values (%d, %d, %f) ;
    """
    year, month = extract_year_and_month(cci.iloc[i].TIME)
    coincident_value = cci.iloc[i].DATA_VALUE

    mystring = (query % (int(year), int(month), float(coincident_value)))
    print(mystring)
    cur.execute(mystring)

 
    Insert into coincident_composite_index (year, month, cci) values (2008, 7, 100.900000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2008, 8, 100.800000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2008, 9, 100.800000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2008, 10, 100.500000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2008, 11, 99.700000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2008, 12, 98.500000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2009, 1, 97.800000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2009, 2, 97.700000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2009, 3, 97.700000) ;
    
 
    Insert into coincident_composite_index (year, month, cci) values (2009, 4, 97.900000) ;
    
 
 

In [75]:
# 확인
query = """ 
select * from coincident_composite_index ;
"""
cur.execute(query)
tmp = pd.DataFrame(list(cur.fetchall()))
tmp

,0,1,2,3
0,1,2008,7,100.9
1,2,2008,8,100.8
2,3,2008,9,100.8
3,4,2008,10,100.5
4,5,2008,11,99.7
...,...,...,...,...
141,142,2020,4,97.6
142,143,2020,5,96.8
143,144,2020,6,97.0
144,145,2020,7,97.2


In [76]:
conn.commit()
cur.close()
conn.close()